In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/train.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tf-roberta/pretrained-roberta-base.h5
/kaggle/input/tf-roberta/config-roberta-base.json
/kaggle/input/tf-roberta/vocab-roberta-base.json
/kaggle/input/tf-roberta/merges-roberta-base.txt
/kaggle/input/tensorflow-roberta-0-705/v0-roberta-2.h5
/kaggle/input/tensorflow-roberta-0-705/v0-roberta-3.h5
/kaggle/input/tensorflow-roberta-0-705/v0-roberta-1.h5
/kaggle/input/tensorflow-roberta-0-705/v0-roberta-4.h5
/kaggle/input/tensorflow-roberta-0-705/__results__.html
/kaggle/input/tensorflow-roberta-0-705/submission.csv
/kaggle/input/tensorflow-roberta-0-705/__resultx__.html
/kaggle/input/tensorflow-roberta-0-705/__notebook__.ipynb
/kaggle/input/tensorflow-roberta-0-705/v0-roberta-0.h5
/kaggle/input/tensorflow-roberta-0-705/__output__.json
/kaggle/input/tensorflow-roberta-0-705/custom.css


In [3]:
import pandas as pd, numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will

Using TensorFlow backend


Loading custom CUDA kernels...
Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu118/cuda_kernel...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/cuda_kernel/build.ninja...
Building extension module cuda_kernel...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/4] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=cuda_kernel -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1016\" -isystem /opt/conda/lib/python3.10/site-packages/torch/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/lib/python3.10/site-packages/torch/include/TH -isystem /opt/conda/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /opt/conda/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=1 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_60,code=compute_60 -gencode=arch=compute_60,code=sm_60 --compiler-options '-fPIC' -std=c++17 -c /opt/conda/lib/python3.10/site-packages/transformers/kernels/mra/cuda_kernel.cu -o cuda_kernel.cuda.o 
[2/4] c++ -MMD -MF torch_extension.o.d -DTORC

Loading extension module cuda_kernel...


TF version 2.13.0


In [4]:
import tokenizers

# Print the version of the tokenizers library
print(tokenizers.__version__)



0.14.1


In [5]:
import tokenizers

MAX_LEN = 96
PATH = '/kaggle/input/tf-roberta'

# Load the tokenizer using the from_file method
tokenizer = tokenizers.ByteLevelBPETokenizer.from_file(
    vocab_filename='/kaggle/input/tf-roberta/vocab-roberta-base.json',
    merges_filename='/kaggle/input/tf-roberta/merges-roberta-base.txt',

    lowercase=True,
    add_prefix_space=True
)

sentiment_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
train = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv').fillna('')


In [6]:
import os

# Print the content of the directory
print(os.listdir('/kaggle/input/tf-roberta/'))




['pretrained-roberta-base.h5', 'config-roberta-base.json', 'vocab-roberta-base.json', 'merges-roberta-base.txt']


In [7]:
ct = train.shape[0]
input_ids = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids = np.zeros((ct,MAX_LEN),dtype='int32')
start_tokens = np.zeros((ct,MAX_LEN),dtype='int32')
end_tokens = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(train.shape[0]):
    
    # FIND OVERLAP
    text1 = " "+" ".join(train.loc[k,'text'].split())
    text2 = " ".join(train.loc[k,'selected_text'].split())
    idx = text1.find(text2)
    chars = np.zeros((len(text1)))
    chars[idx:idx+len(text2)]=1
    if text1[idx-1]==' ': chars[idx-1] = 1 
    enc = tokenizer.encode(text1) 
        
    # ID_OFFSETS
    offsets = []; idx=0
    for t in enc.ids:
        w = tokenizer.decode([t])
        offsets.append((idx,idx+len(w)))
        idx += len(w)
    
    # START END TOKENS
    toks = []
    for i,(a,b) in enumerate(offsets):
        sm = np.sum(chars[a:b])
        if sm>0: toks.append(i) 
        
    s_tok = sentiment_id[train.loc[k,'sentiment']]
    input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask[k,:len(enc.ids)+5] = 1
    if len(toks)>0:
        start_tokens[k,toks[0]+1] = 1
        end_tokens[k,toks[-1]+1] = 1

In [8]:
test = pd.read_csv('../input/tweet-sentiment-extraction/test.csv').fillna('')

ct = test.shape[0]
input_ids_t = np.ones((ct,MAX_LEN),dtype='int32')
attention_mask_t = np.zeros((ct,MAX_LEN),dtype='int32')
token_type_ids_t = np.zeros((ct,MAX_LEN),dtype='int32')

for k in range(test.shape[0]):
        
    # INPUT_IDS
    text1 = " "+" ".join(test.loc[k,'text'].split())
    enc = tokenizer.encode(text1)                
    s_tok = sentiment_id[test.loc[k,'sentiment']]
    input_ids_t[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
    attention_mask_t[k,:len(enc.ids)+5] = 1

In [15]:
# from tensorflow.keras import layers as L
# from tensorflow.keras.models import Model
# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam

# from tensorflow.keras.layers import MultiHeadAttention


# #multi-head self-attention model 

# def build_model():
#     ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
#     att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
#     tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)

#     # Load RoBERTa model
#     config = RobertaConfig.from_json_file('/kaggle/input/tf-roberta/config-roberta-base.json')
#     bert_model = TFRobertaModel.from_pretrained('/kaggle/input/tf-roberta/pretrained-roberta-base.h5', config=config)
#     x = bert_model(ids, attention_mask=att, token_type_ids=tok)

#     # Add multi-head self-attention
#     key_dim = 128  # You can adjust the key_dim
#     attention_heads = 8  # You can adjust the number of attention heads
#     x_att = MultiHeadAttention(num_heads=attention_heads, key_dim=key_dim)(x[0], x[0], x[0])  # Use MultiHeadAttention
#     x = tf.keras.layers.Add()([x[0], x_att])

#     # Continue with your model
#     x1 = tf.keras.layers.Dropout(0.1)(x)
#     x1 = tf.keras.layers.Conv1D(1, 1)(x1)
#     x1 = tf.keras.layers.Flatten()(x1)
#     x1 = tf.keras.layers.Activation('softmax')(x1)

#     x2 = tf.keras.layers.Dropout(0.1)(x)
#     x2 = tf.keras.layers.Conv1D(1, 1)(x2)
#     x2 = tf.keras.layers.Flatten()(x2)
#     x2 = tf.keras.layers.Activation('softmax')(x2)

#     model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1, x2])
#     optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#     return model





In [10]:

# #original- baseline model

# def build_model():
#     ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
#     att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
#     tok = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32)
#     config = RobertaConfig.from_json_file('/kaggle/input/tf-roberta/config-roberta-base.json')
#     #config = RobertaConfig.from_pretrained(PATH+'config-roberta-base.json')
#     #bert_model = TFRobertaModel.from_pretrained(PATH+'pretrained-roberta-base.h5',config=config)
#     bert_model = TFRobertaModel.from_pretrained('/kaggle/input/tf-roberta/pretrained-roberta-base.h5',config=config)
#     x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
#     x1 = tf.keras.layers.Dropout(0.1)(x[0]) 
#     x1 = tf.keras.layers.Conv1D(1,1)(x1)
#     x1 = tf.keras.layers.Flatten()(x1)
#     x1 = tf.keras.layers.Activation('softmax')(x1)
    
#     x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
#     x2 = tf.keras.layers.Conv1D(1,1)(x2)
#     x2 = tf.keras.layers.Flatten()(x2)
#     x2 = tf.keras.layers.Activation('softmax')(x2)

#     model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
#     optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#     return model

In [19]:
# #dynamic attention model 
# import tensorflow as tf
# from tensorflow.keras.layers import Layer

# class DynamicAttention(Layer):
#     def __init__(self, key_dim, value_dim, **kwargs):
#         super(DynamicAttention, self).__init__(**kwargs)
#         self.key_dim = key_dim
#         self.value_dim = value_dim
#         self.dense_query = tf.keras.layers.Dense(self.key_dim, activation='relu')
#         self.dense_key = tf.keras.layers.Dense(self.key_dim, activation='relu')
#         self.linear_projection_query = tf.keras.layers.Dense(self.value_dim)
#         self.linear_projection_key = tf.keras.layers.Dense(self.value_dim)
#         self.attention = tf.keras.layers.Attention(use_scale=True)

#     def call(self, inputs):
#         query, key, value = inputs

#         # Ensure the dimensions match
#         processed_query = self.dense_query(query)
#         processed_key = self.dense_key(key)

#         # Add linear projections to match the attention mechanism dimensions
#         linear_projection_query = self.linear_projection_query(processed_query)
#         linear_projection_key = self.linear_projection_key(processed_key)

#         context = self.attention([linear_projection_query, linear_projection_key, value])

#         return context


# from tensorflow.keras.layers import Input, Dropout, Conv1D, Flatten, Activation
# from transformers import RobertaConfig, TFRobertaModel

# #dynamic attention model 

# def build_model():
#     ids = Input((MAX_LEN,), dtype=tf.int32)
#     att = Input((MAX_LEN,), dtype=tf.int32)
#     tok = Input((MAX_LEN,), dtype=tf.int32)

#     config = RobertaConfig.from_json_file('/kaggle/input/tf-roberta/config-roberta-base.json')
#     bert_model = TFRobertaModel.from_pretrained('/kaggle/input/tf-roberta/pretrained-roberta-base.h5', config=config)
    
#     x = bert_model(ids, attention_mask=att, token_type_ids=tok)

#     key_dim = 768
#     value_dim = 768  # You can adjust this value
#     dynamic_attention = DynamicAttention(key_dim=key_dim, value_dim=value_dim)([x[0], x[0], x[0]])  # Dynamic attention


#     x_att = tf.keras.layers.Add()([x[0], dynamic_attention])

#     x1 = Dropout(0.1)(x_att)
#     x1 = Conv1D(1, 1)(x1)
#     x1 = Flatten()(x1)
#     x1 = Activation('softmax')(x1)

#     x2 = Dropout(0.1)(x_att)
#     x2 = Conv1D(1, 1)(x2)
#     x2 = Flatten()(x2)
#     x2 = Activation('softmax')(x2)

#     model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1, x2])
#     optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#     return model

In [31]:

# import tensorflow as tf
# from tensorflow.keras.layers import Layer


# #muti-head dynamic attention model
# class MultiHeadDynamicAttention(Layer):
#     def __init__(self, key_dim, value_dim, num_heads, **kwargs):
#         super(MultiHeadDynamicAttention, self).__init__(**kwargs)
#         self.key_dim = key_dim
#         self.value_dim = value_dim
#         self.num_heads = num_heads

#         self.dense_query = [tf.keras.layers.Dense(self.key_dim, activation='relu') for _ in range(num_heads)]
#         self.dense_key = [tf.keras.layers.Dense(self.key_dim, activation='relu') for _ in range(num_heads)]
#         self.linear_projection_query = [tf.keras.layers.Dense(self.value_dim) for _ in range(num_heads)]
#         self.linear_projection_key = [tf.keras.layers.Dense(self.value_dim) for _ in range(num_heads)]
#         self.attention = tf.keras.layers.Attention(use_scale=True)

#     def call(self, inputs):
#         query, key, value = inputs

#         # Ensure the dimensions match
#         processed_queries = [dense(query) for dense in self.dense_query]
#         processed_keys = [dense(key) for dense in self.dense_key]

#         # Add linear projections to match the attention mechanism dimensions
# #         linear_projections_query = [linear(processed_query) for linear, processed_query in zip(self.linear_projection_query, processed_queries)]
# #         linear_projections_key = [linear(processed_key) for linear, processed_key in zip(self.linear_projection_key, processed_keys)]
#         linear_projections_query = [dense_linear(processed_query) for dense_linear, processed_query in zip(self.linear_projection_query, processed_queries)]
#         linear_projections_key = [dense_linear(processed_key) for dense_linear, processed_key in zip(self.linear_projection_key, processed_keys)]


#         # Perform attention for each head
#         attentions = [self.attention([linear_projection_query, linear_projection_key, value]) for linear_projection_query, linear_projection_key in zip(linear_projections_query, linear_projections_key)]

#         # Concatenate or average the outputs from different heads
#         concatenated_attentions = tf.concat(attentions, axis=-1)
#         averaged_attention = tf.reduce_mean(attentions, axis=0)

#         return averaged_attention  # You can return concatenated_attentions if you prefer concatenation

    


# #muti-head dynamic attention model 

# from tensorflow.keras.layers import Input, Dropout, Conv1D, Flatten, Activation
# from transformers import RobertaConfig, TFRobertaModel
# from tensorflow.keras.optimizers.schedules import ExponentialDecay

# def build_model():
#     ids = Input((MAX_LEN,), dtype=tf.int32)
#     att = Input((MAX_LEN,), dtype=tf.int32)
#     tok = Input((MAX_LEN,), dtype=tf.int32)

#     config = RobertaConfig.from_json_file('/kaggle/input/tf-roberta/config-roberta-base.json')
#     bert_model = TFRobertaModel.from_pretrained('/kaggle/input/tf-roberta/pretrained-roberta-base.h5', config=config)
    
#     x = bert_model(ids, attention_mask=att, token_type_ids=tok)

#     key_dim = 768
#     value_dim = 768  # You can adjust this value
#     num_heads = 4  # You can adjust the number of heads
#     dynamic_attention = MultiHeadDynamicAttention(key_dim=key_dim, value_dim=value_dim, num_heads=num_heads)([x[0], x[0], x[0]])



#     x_att = tf.keras.layers.Add()([x[0], dynamic_attention])

#     x1 = Dropout(0.1)(x_att)
#     x1 = Conv1D(1, 1)(x1)
#     x1 = Flatten()(x1)
#     x1 = Activation('softmax')(x1)

#     x2 = Dropout(0.1)(x_att)
#     x2 = Conv1D(1, 1)(x2)
#     x2 = Flatten()(x2)
#     x2 = Activation('softmax')(x2)

#     model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1, x2])
#     optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
#     model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#     return model

In [38]:

import tensorflow as tf
from tensorflow.keras.layers import Layer


#muti-head dynamic attention model
class MultiHeadDynamicAttention(Layer):
    def __init__(self, key_dim, value_dim, num_heads, **kwargs):
        super(MultiHeadDynamicAttention, self).__init__(**kwargs)
        self.key_dim = key_dim
        self.value_dim = value_dim
        self.num_heads = num_heads

        self.dense_query = [tf.keras.layers.Dense(self.key_dim, activation='relu') for _ in range(num_heads)]
        self.dense_key = [tf.keras.layers.Dense(self.key_dim, activation='relu') for _ in range(num_heads)]
        self.linear_projection_query = [tf.keras.layers.Dense(self.value_dim) for _ in range(num_heads)]
        self.linear_projection_key = [tf.keras.layers.Dense(self.value_dim) for _ in range(num_heads)]
        self.attention = tf.keras.layers.Attention(use_scale=True)

    def call(self, inputs):
        query, key, value = inputs

        # Ensure the dimensions match
        processed_queries = [dense(query) for dense in self.dense_query]
        processed_keys = [dense(key) for dense in self.dense_key]

        # Add linear projections to match the attention mechanism dimensions
#         linear_projections_query = [linear(processed_query) for linear, processed_query in zip(self.linear_projection_query, processed_queries)]
#         linear_projections_key = [linear(processed_key) for linear, processed_key in zip(self.linear_projection_key, processed_keys)]
        linear_projections_query = [dense_linear(processed_query) for dense_linear, processed_query in zip(self.linear_projection_query, processed_queries)]
        linear_projections_key = [dense_linear(processed_key) for dense_linear, processed_key in zip(self.linear_projection_key, processed_keys)]


        # Perform attention for each head
        attentions = [self.attention([linear_projection_query, linear_projection_key, value]) for linear_projection_query, linear_projection_key in zip(linear_projections_query, linear_projections_key)]

        # Concatenate or average the outputs from different heads
        concatenated_attentions = tf.concat(attentions, axis=-1)
        averaged_attention = tf.reduce_mean(attentions, axis=0)

        return concatenated_attentions  # You can return concatenated_attentions if you prefer concatenation

    


#muti-head dynamic attention model 

from tensorflow.keras.layers import Input, Dropout, Conv1D, Flatten, Activation
from transformers import RobertaConfig, TFRobertaModel
from tensorflow.keras.optimizers.schedules import ExponentialDecay

def build_model():
    ids = Input((MAX_LEN,), dtype=tf.int32)
    att = Input((MAX_LEN,), dtype=tf.int32)
    tok = Input((MAX_LEN,), dtype=tf.int32)

    config = RobertaConfig.from_json_file('/kaggle/input/tf-roberta/config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained('/kaggle/input/tf-roberta/pretrained-roberta-base.h5', config=config)
    
    x = bert_model(ids, attention_mask=att, token_type_ids=tok)

    key_dim = 768
    value_dim = 768  # You can adjust this value
    num_heads = 4  # You can adjust the number of heads
    dynamic_attention = MultiHeadDynamicAttention(key_dim=key_dim, value_dim=value_dim, num_heads=num_heads)([x[0], x[0], x[0]])



    #x_att = tf.keras.layers.Add()([x[0], dynamic_attention])
    x_att = tf.keras.layers.Concatenate(axis=-1)([x[0], dynamic_attention])


    x1 = Dropout(0.1)(x_att)
    x1 = Conv1D(1, 1)(x1)
    x1 = Flatten()(x1)
    x1 = Activation('softmax')(x1)

    x2 = Dropout(0.1)(x_att)
    x2 = Conv1D(1, 1)(x2)
    x2 = Flatten()(x2)
    x2 = Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1, x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [39]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    if (len(a)==0) & (len(b)==0): return 0.5
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [40]:
# jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
# oof_start = np.zeros((input_ids.shape[0], MAX_LEN))
# oof_end = np.zeros((input_ids.shape[0], MAX_LEN))
# preds_start = np.zeros((input_ids_t.shape[0], MAX_LEN))
# preds_end = np.zeros((input_ids_t.shape[0], MAX_LEN))

# fold = 0  # Specify the fold you want to train

# print('#' * 25)
# print('### FOLD %i' % (fold + 1))
# print('#' * 25)

# K.clear_session()
# model = build_model()

# sv = tf.keras.callbacks.ModelCheckpoint(
#     '%s-roberta-%i.h5' % (VER, fold), monitor='val_loss', verbose=1, save_best_only=True,
#     save_weights_only=True, mode='auto', save_freq='epoch')

# # Use StratifiedKFold to get a single split for the specified fold
# idxT, idxV = next(iter(StratifiedKFold(n_splits=5, shuffle=True, random_state=777).split(input_ids, train.sentiment.values)))

# model.fit(
#     [input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]],
#     [start_tokens[idxT,], end_tokens[idxT,]],
#     epochs=1, batch_size=32, verbose=DISPLAY, callbacks=[sv],
#     validation_data=(
#         [input_ids[idxV,], attention_mask[idxV,], token_type_ids[idxV,]],
#         [start_tokens[idxV,], end_tokens[idxV,]]
#     )
# )




# print('Loading model...')
# model.load_weights('%s-roberta-%i.h5' % (VER, fold))

# print('Predicting OOF...')
# oof_start[idxV,], oof_end[idxV,] = model.predict(
#     [input_ids[idxV,], attention_mask[idxV,], token_type_ids[idxV,]],
#     verbose=DISPLAY
# )

# print('Predicting Test...')
# preds = model.predict([input_ids_t, attention_mask_t, token_type_ids_t], verbose=DISPLAY)
# preds_start += preds[0]
# preds_end += preds[1]

# # Display Jaccard for the single fold
# all = []
# for k in idxV:
#     a = np.argmax(oof_start[k,])
#     b = np.argmax(oof_end[k,])
#     if a > b:
#         st = train.loc[k, 'text']
#     else:
#         text1 = " " + " ".join(train.loc[k, 'text'].split())
#         enc = tokenizer.encode(text1)
#         st = tokenizer.decode(enc.ids[a - 1:b])
#     all.append(jaccard(st, train.loc[k, 'selected_text']))

# print('>>>> FOLD %i Jaccard =' % (fold + 1), np.mean(all))


In [41]:
#5 fold 3 epochs trainning
jac = []; VER='v0'; DISPLAY=1 # USE display=1 FOR INTERACTIVE
oof_start = np.zeros((input_ids.shape[0],MAX_LEN))
oof_end = np.zeros((input_ids.shape[0],MAX_LEN))
preds_start = np.zeros((input_ids_t.shape[0],MAX_LEN))
preds_end = np.zeros((input_ids_t.shape[0],MAX_LEN))

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=777)
for fold,(idxT,idxV) in enumerate(skf.split(input_ids,train.sentiment.values)):

    print('#'*25)
    print('### FOLD %i'%(fold+1))
    print('#'*25)
    
    K.clear_session()
    model = build_model()
        
    sv = tf.keras.callbacks.ModelCheckpoint(
        '%s-roberta-%i.h5'%(VER,fold), monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=True, mode='auto', save_freq='epoch')
        
    model.fit([input_ids[idxT,], attention_mask[idxT,], token_type_ids[idxT,]], [start_tokens[idxT,], end_tokens[idxT,]], 
        epochs=3, batch_size=32, verbose=DISPLAY, callbacks=[sv],
        validation_data=([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]], 
        [start_tokens[idxV,], end_tokens[idxV,]]))
    
    print('Loading model...')
    model.load_weights('%s-roberta-%i.h5'%(VER,fold))
    
    print('Predicting OOF...')
    oof_start[idxV,],oof_end[idxV,] = model.predict([input_ids[idxV,],attention_mask[idxV,],token_type_ids[idxV,]],verbose=DISPLAY)
    
    print('Predicting Test...')
    preds = model.predict([input_ids_t,attention_mask_t,token_type_ids_t],verbose=DISPLAY)
    preds_start += preds[0]/skf.n_splits
    preds_end += preds[1]/skf.n_splits
    
    # DISPLAY FOLD JACCARD
    all = []
    for k in idxV:
        a = np.argmax(oof_start[k,])
        b = np.argmax(oof_end[k,])
        if a>b: 
            st = train.loc[k,'text'] # IMPROVE CV/LB with better choice here
        else:
            text1 = " "+" ".join(train.loc[k,'text'].split())
            enc = tokenizer.encode(text1)
            st = tokenizer.decode(enc.ids[a-1:b])
        all.append(jaccard(st,train.loc[k,'selected_text']))
    jac.append(np.mean(all))
    print('>>>> FOLD %i Jaccard ='%(fold+1),np.mean(all))
    print()

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


#########################
### FOLD 1
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /kaggle/input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
687/687 [==============================] - ETA: 0s - loss: 2.3619 - activation_loss: 1.1604 - activation_1_loss: 1.2016
Epoch 1: val_loss improved from inf to 1.74552, saving model to v0-roberta-0.h5
687/687 [==============================] - 343s 431ms/step - loss: 2.3619 - activation_loss: 1.1604 - activation_1_loss: 1.2016 - val_loss: 1.7455 - val_activation_loss: 0.8913 - val_activation_1_loss: 0.8542
Epoch 2/3
687/687 [==============================] - ETA: 0s - loss: 1.7186 - activation_loss: 0.8719 - activation_1_loss: 0.8468
Epoch 2: val_loss improved from 1.74552 to 1.72995, saving model to v0-roberta-0.h5
687/687 [==============================] - 290s 422ms/step - loss: 1.7186 - activation_loss: 0.8719 - activation_1_loss: 0.8468 - val_loss: 1.7300 - val_activation_loss: 0.8827 - val_activation_1_loss: 0.8473
Epoch 3/3
687/687 [==============================] - ETA: 0s - loss: 1.5804 - activation_loss: 0.8169 - activation_1_loss: 0.7635
Epoch 3: val_loss improved f

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


...

Epoch 1/3
688/688 [==============================] - ETA: 0s - loss: 2.3624 - activation_loss: 1.1629 - activation_1_loss: 1.1995
Epoch 1: val_loss improved from inf to 1.78092, saving model to v0-roberta-3.h5
688/688 [==============================] - 363s 450ms/step - loss: 2.3624 - activation_loss: 1.1629 - activation_1_loss: 1.1995 - val_loss: 1.7809 - val_activation_loss: 0.8851 - val_activation_1_loss: 0.8958
Epoch 2/3
688/688 [==============================] - ETA: 0s - loss: 1.6798 - activation_loss: 0.8580 - activation_1_loss: 0.8218
Epoch 2: val_loss improved from 1.78092 to 1.62370, saving model to v0-roberta-3.h5
688/688 [==============================] - 303s 440ms/step - loss: 1.6798 - activation_loss: 0.8580 - activation_1_loss: 0.8218 - val_loss: 1.6237 - val_activation_loss: 0.8249 - val_activation_1_loss: 0.7988
Epoch 3/3
688/688 [==============================] - ETA: 0s - loss: 1.5250 - activation_loss: 0.7895 - activation_1_loss: 0.7355
Epoch 3: val_loss did not im

loading weights file /kaggle/input/tf-roberta/pretrained-roberta-base.h5


>>>> FOLD 4 Jaccard = 0.7053618196174949

#########################
### FOLD 5
#########################


All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /kaggle/input/tf-roberta/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/3
688/688 [==============================] - ETA: 0s - loss: 2.4660 - activation_loss: 1.1895 - activation_1_loss: 1.2765
Epoch 1: val_loss improved from inf to 1.83832, saving model to v0-roberta-4.h5
688/688 [==============================] - 362s 449ms/step - loss: 2.4660 - activation_loss: 1.1895 - activation_1_loss: 1.2765 - val_loss: 1.8383 - val_activation_loss: 0.8819 - val_activation_1_loss: 0.9564
Epoch 2/3
688/688 [==============================] - ETA: 0s - loss: 1.9521 - activation_loss: 0.9092 - activation_1_loss: 1.0429
Epoch 2: val_loss improved from 1.83832 to 1.67398, saving model to v0-roberta-4.h5
688/688 [==============================] - 303s 440ms/step - loss: 1.9521 - activation_loss: 0.9092 - activation_1_loss: 1.0429 - val_loss: 1.6740 - val_activation_loss: 0.8324 - val_activation_1_loss: 0.8415
Epoch 3/3
688/688 [==============================] - ETA: 0s - loss: 1.6489 - activation_loss: 0.8294 - activation_1_loss: 0.8194
Epoch 3: val_loss improved f

In [42]:
print('>>>> OVERALL 5Fold CV Jaccard =',np.mean(jac))


>>>> OVERALL 5Fold CV Jaccard = 0.7018045736467972
